#**Step 1: installazione delle librerie**

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops sentencepiece

In [ ]:
!pip install -q accelerate
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl
!pip install huggingface_hub
!pip install accelerate
!pip install --upgrade torch

#**Step 2: caricamento delle librerie**

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

#import wandb, platform, warnings
import warnings

import pandas as pd
from datasets import Dataset
import re
import random

In [ ]:
data_path_gem_models = "..."
data_path_data = "..."

#**Step 3: impostazione delle variabili**

In [ ]:
import random

percentuale_dataset = 0.2

# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"

# Number of training epochs
num_train_epochs = 1

# Fine-tuned model name
new_model = f"mistral_7b_{num_train_epochs}epoch"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4: caricamento del dataset**

In [ ]:
# carico il dataset
trainset = pd.read_csv(f'{data_path_data}/train.csv')
devset = pd.read_csv(f'{data_path_data}/dev.csv')

# ottengo un sottoinsieme del dataset di partenza
subset_trainset = trainset.sample(frac=percentuale_dataset, random_state=42)
subset_trainset.head()

In [ ]:
print(len(trainset))

In [ ]:
print(len(subset_trainset))

In [ ]:
# ottengo un sottoinsieme del dataset di partenza
subset_devset = devset.sample(frac=percentuale_dataset, random_state=42)
subset_devset.head()

In [ ]:
print(len(subset_devset))

In [ ]:
mistral_trainset = []
for index, row in subset_trainset.iterrows():
    triple = row['triple']
    sentence = row['sentence']

    mistral_trainset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

mistral_trainset = pd.DataFrame(mistral_trainset, columns=['instr'])
mistral_trainset.head()

In [ ]:
mistral_trainset = Dataset.from_pandas(mistral_trainset)

In [ ]:
mistral_devset = []
for index, row in subset_devset.iterrows():
    triple = row['triple']
    sentence = row['sentence']

    mistral_devset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

mistral_devset = pd.DataFrame(mistral_devset, columns=['instr'])
mistral_devset.head()

In [ ]:
mistral_devset = Dataset.from_pandas(mistral_devset)

#**Step 5: caricamento del modello**

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
login(token="...")

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

#**Step 6: fine-tuning del modello**

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy="steps",  # Calcolo della validation loss ad ogni step
    save_strategy="steps",  # Checkpoint del modello ad ogni step
    save_total_limit=3,  # Limita il numero di checkpoint salvati
    load_best_model_at_end=True,  # Carica il modello migliore al termine dell'addestramento
    metric_for_best_model="eval_loss",  # Sceglie la metrica per determinare il miglior modello
    greater_is_better=False,  # Indica se un valore più alto della metrica è migliore o no
    eval_steps=50,  # Numero di passaggi prima di valutare il modello
    logging_steps= 50,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=mistral_trainset,
    eval_dataset=mistral_devset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="instr",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

# Train model
trainer.train()

In [ ]:
# Sostituisci "path_to_save" con il percorso desiderato nel tuo drive
trainer.save_model(f'{data_path_gem_models}/{new_model}')

#**Step 7: generazione frasi del modello creato**

In [ ]:
gc.collect()

In [ ]:
import gc
gc.collect()
gc.collect()

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="torch.utils.checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, message="None of the inputs have requires_grad=True")

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,  # Assicurati che FP16 sia supportato dal modello
    device_map=device_map,
)

In [ ]:
#model = PeftModel.from_pretrained(base_model, f'{data_path_gem_models}/{new_model}')
model = PeftModel.from_pretrained(base_model, f'{data_path_gem_models}/mistral_7b_1epoch')
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Decoding standard

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text(triple, actual):
    max_length = int(len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {actual} [/ANW]") * 1.2)

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length, temperature=0.1)
    result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

    if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
        generated_text = result[0]['generated_text']

        # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
        match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

        if match:
            content_within_anw_tag = match.group(1)
            text = content_within_anw_tag
        else:
            text = generated_text
    else:
        text = "Il risultato della pipeline è nullo."

    return text

In [ ]:
testset = pd.read_csv(f'{data_path_data}/test.csv')

# ottengo un sottoinsieme formato da 1000 elementi
perc_d = 1000 / len(testset)
subset_testset = testset.sample(frac=perc_d, random_state=42)
print(len(subset_testset))

# test
X_test = subset_testset['triple'].values.tolist()
y_test = subset_testset['sentence'].values.tolist()

In [ ]:
subset_testset.head()

In [ ]:
triples = []
predictions = []
actuals = []

i = 0
for tripla in X_test:
    actual = y_test[i]
    print(f'======== STEP {i} ======== ')
    print(f'Tripla: {tripla}')
    print(f'Actual: {actual}')

    prediction = generate_text(tripla, actual)

    triples.append(tripla)
    actuals.append(actual)
    predictions.append(prediction)


    print(f'Prediction: {prediction}')
    print('\n')

    i = i + 1

    df = pd.DataFrame(list(zip(triples, predictions, actuals)), columns =['triples', 'predictions', 'actuals'])
    df.to_csv(f'{data_path_gem_models}/{new_model}_decoding.csv', index=False)

## Decoding per sentence aggregation

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text_sa(triple, sentence_len):
  max_length = len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] [/ANW]") + sentence_len

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length, temperature=0.1)
  result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

  if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
      generated_text = result[0]['generated_text']

      # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
      match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

      if match:
          content_within_anw_tag = match.group(1)
          text = content_within_anw_tag
      else:
          text = generated_text

  else:
      text = "Il risultato della pipeline è nullo."

  return text

In [ ]:
testset = pd.read_csv(f'{data_path_data}/test_sga_1.csv')

subset_testset = testset

subset_testset.head()

In [ ]:
len(subset_testset)

In [ ]:
current_id = -1
rows = []
for i in range(len(subset_testset)):
  id = subset_testset['id'][i]
  triples = subset_testset['triples'][i]
  data_unit = subset_testset['data_unit'][i]
  actual = subset_testset['actual'][i]
  shape = subset_testset['shape'][i]
  shape_type = subset_testset['shape_type'][i]
  local_size = subset_testset['local_size'][i]
  size = subset_testset['size'][i]

  if current_id == -1 or current_id != id:
    if current_id != -1:
      rows.append(row)

    row = {
        'id' : id,
        'data_unit' : data_unit,
        'sentences': [],
        'actual' : actual,
        'shape': shape,
        'shape_type': shape_type,
        'size': size,
    }

  prediction = generate_text_sa(triples, len(actual))
  row['sentences'].append(prediction)
  current_id = id

  print(f"======== Step {i} ========")
  print(f"tripla: {triples}")
  print(f"predizione: {prediction}")
  print('\n')

  # Crea un DataFrame da rows
  df = pd.DataFrame(rows)
  #df.to_csv(f'{data_path_gem_models}/{new_model}_decoding_to_aggregate.csv', index=False)
  df.to_csv(f'{data_path_gem_models}/mistral_7b_test_sga_2.csv', index=False)